## Test bed for project code

In [1]:
!ls

README.md              process-data.ipynb     requests_scraper.ipynb
__pycache__            process_data.py        scrapers.py
analyze-data.ipynb     processed_data         test-bed.ipynb
backup_data            project-3-work.ipynb   user-graph.ipynb
data-rescue.ipynb      raw_data
presentation           reddit_scrape.py


In [6]:
import pandas as pd
import datetime

pd.options.display.max_columns = None

In [4]:
chunks = pd.read_csv('../processed_data/test_data.csv', chunksize=10000)
df = pd.concat(chunks, ignore_index=True)

In [16]:
df['compound'].sum()

6.4019000000000004

In [6]:
content = df['content']

In [7]:
def outside_content(content):
    outside = 0
    if content[1] != 'r':
        outside = 1
    return outside

df['outside_content'] = df['content'].apply(outside_content)

In [8]:
def is_question(title):
    question = 0
    if '?' in title:
        question = 1
    return question

df['is_question'] = df['title'].apply(is_question)

In [9]:
df[['content', 'outside_content']].head()

,content,outside_content
0,https://i.redd.it/enqtmpug0suz.jpg,1
1,/r/NorthCarolina/comments/79l16l/living_in_nc/,0
2,https://www.youtube.com/watch?v=r06GHx795LU&fe...,1
3,http://www.foxcarolina.com/story/36704597/warr...,1
4,http://abc11.com/mother-charged-with-killing-s...,1


In [10]:
df['scrape_time'] = df['scrape_time'].apply(datetime.datetime.fromtimestamp)

In [11]:
df['post_time'] = df['post_time']/1000
df['post_time'] = df['post_time'].apply(datetime.datetime.fromtimestamp)

df['elapsed_time'] = df['scrape_time'] - df['post_time']
df['minutes_since_post'] = df['elapsed_time'].dt.total_seconds()/60

In [12]:
df.head(3)

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title,has_emoji,outside_content,is_question,elapsed_time,minutes_since_post
0,SauceOfTheBoss,8010.0,19,https://i.redd.it/enqtmpug0suz.jpg,0,11126.0,/r/NorthCarolina/comments/79gndn/pilot_mountai...,t3_79gndn,2017-10-29 10:05:39,294.0,2017-10-30 06:10:43.413661,r/NorthCarolina,3817.0,20285.0,Pilot Mountain- 10/27/17 (i.redd.it),0,1,0,20:05:04.413661,1205.073561
1,WoodstockIV79,181.0,4,/r/NorthCarolina/comments/79l16l/living_in_nc/,0,19.0,/r/NorthCarolina/comments/79l16l/living_in_nc/,t3_79l16l,2017-10-29 22:45:16,5.0,2017-10-30 06:10:43.942422,r/NorthCarolina,3817.0,20285.0,discussionLiving in NC (self.NorthCarolina),0,0,0,07:25:27.942422,445.465707
2,newkid261,22.0,0,https://www.youtube.com/watch?v=r06GHx795LU&fe...,0,5.0,/r/NorthCarolina/comments/79m4vh/fightung_for_...,t3_79m4vh,2017-10-30 03:03:02,1.0,2017-10-30 06:10:44.240265,r/NorthCarolina,3817.0,20285.0,Fightung for north carolina (youtube.com),0,1,0,03:07:42.240265,187.704004


In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()

sentiment = df['title'].apply(sia.polarity_scores)

sent = pd.DataFrame(list(sentiment))

df_2 = df.join(sent)

In [18]:
df_2.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,...,title,has_emoji,outside_content,is_question,elapsed_time,minutes_since_post,compound,neg,neu,pos
0,SauceOfTheBoss,8010.0,19,https://i.redd.it/enqtmpug0suz.jpg,0,11126.0,/r/NorthCarolina/comments/79gndn/pilot_mountai...,t3_79gndn,2017-10-29 10:05:39,294.0,...,Pilot Mountain- 10/27/17 (i.redd.it),0,1,0,20:05:04.413661,1205.073561,0.0000,0.00,1.000,0.000
1,WoodstockIV79,181.0,4,/r/NorthCarolina/comments/79l16l/living_in_nc/,0,19.0,/r/NorthCarolina/comments/79l16l/living_in_nc/,t3_79l16l,2017-10-29 22:45:16,5.0,...,discussionLiving in NC (self.NorthCarolina),0,0,0,07:25:27.942422,445.465707,0.0000,0.00,1.000,0.000
2,newkid261,22.0,0,https://www.youtube.com/watch?v=r06GHx795LU&fe...,0,5.0,/r/NorthCarolina/comments/79m4vh/fightung_for_...,t3_79m4vh,2017-10-30 03:03:02,1.0,...,Fightung for north carolina (youtube.com),0,1,0,03:07:42.240265,187.704004,0.0000,0.00,1.000,0.000
3,Cynner,15047.0,11,http://www.foxcarolina.com/story/36704597/warr...,0,17677.0,/r/NorthCarolina/comments/79gab7/warren_wilson...,t3_79gab7,2017-10-29 08:43:31,22.0,...,Warren Wilson College to offer free tuition to...,0,1,0,21:27:13.721249,1287.228687,0.5106,0.00,0.769,0.231
4,JohnKimble111,37146.0,0,http://abc11.com/mother-charged-with-killing-s...,0,505294.0,/r/NorthCarolina/comments/79kux3/mother_charge...,t3_79kux3,2017-10-29 22:11:08,2.0,...,Mother charged with killing son at Fayettevill...,0,1,0,07:59:37.082165,479.618036,-0.7351,0.47,0.530,0.000


In [15]:
import re

In [28]:
df['title'] = [re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", title)).replace('()', '') for title in df['title'].values]



In [20]:
df.shape



(1024552, 20)

In [27]:
df.dropna('title', inplace=True)

ValueError: No axis named title for object type <class 'pandas.core.frame.DataFrame'>

In [29]:
df.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title,has_emoji,outside_content,is_question,elapsed_time,minutes_since_post
0,SauceOfTheBoss,8010.0,19,https://i.redd.it/enqtmpug0suz.jpg,0,11126.0,/r/NorthCarolina/comments/79gndn/pilot_mountai...,t3_79gndn,2017-10-29 10:05:39,294.0,2017-10-30 06:10:43.413661,r/NorthCarolina,3817.0,20285.0,Pilot Mountain- 10/27/17,0,1,0,20:05:04.413661,1205.073561
1,WoodstockIV79,181.0,4,/r/NorthCarolina/comments/79l16l/living_in_nc/,0,19.0,/r/NorthCarolina/comments/79l16l/living_in_nc/,t3_79l16l,2017-10-29 22:45:16,5.0,2017-10-30 06:10:43.942422,r/NorthCarolina,3817.0,20285.0,discussion Living in NC,0,0,0,07:25:27.942422,445.465707
2,newkid261,22.0,0,https://www.youtube.com/watch?v=r06GHx795LU&fe...,0,5.0,/r/NorthCarolina/comments/79m4vh/fightung_for_...,t3_79m4vh,2017-10-30 03:03:02,1.0,2017-10-30 06:10:44.240265,r/NorthCarolina,3817.0,20285.0,Fightung for north carolina,0,1,0,03:07:42.240265,187.704004
3,Cynner,15047.0,11,http://www.foxcarolina.com/story/36704597/warr...,0,17677.0,/r/NorthCarolina/comments/79gab7/warren_wilson...,t3_79gab7,2017-10-29 08:43:31,22.0,2017-10-30 06:10:44.721249,r/NorthCarolina,3817.0,20285.0,Warren Wilson College to offer free tuition to...,0,1,0,21:27:13.721249,1287.228687
4,JohnKimble111,37146.0,0,http://abc11.com/mother-charged-with-killing-s...,0,505294.0,/r/NorthCarolina/comments/79kux3/mother_charge...,t3_79kux3,2017-10-29 22:11:08,2.0,2017-10-30 06:10:45.082165,r/NorthCarolina,3817.0,20285.0,Mother charged with killing son at Fayettevill...,0,1,0,07:59:37.082165,479.618036
